In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials = {
    'IAM_SERVICE_ID': '*****',
    'IBM_API_KEY_ID': '****',
    'ENDPOINT': '****',
    'IBM_AUTH_ENDPOINT': '***',
    'BUCKET': '****',
    'FILE': 'transactions_table.csv'
}

client_don = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])

client_don.download_file(Bucket=credentials['BUCKET'],Key='kin_security.parquet.gzip',Filename='kin_security.parquet.gzip')

# Data understanding & Data Preparation

In [3]:
df = pd.read_parquet('kin_security.parquet.gzip')

/opt/conda/envs/Python36/lib/python3.6/site-packages/pyarrow/pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
/opt/conda/envs/Python36/lib/python3.6/site-packages/pyarrow/pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
/opt/conda/envs/Python36/lib/python3.6/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [4]:
df.head()

,CustomerId,Geography,Gender,HasCrCard,IsActiveMember,EstimatedSalary,application_date,exit_date,birth_date,years_company,age,balance_bank,Products,Score,Desertion
0,15610711,Germany,Female,0,0,167673.37,2017-03-05,2019-11-27,1976-10-21,2.737907,40,1.286445e+05,1,644.500000,0
1,15764170,Germany,Male,1,0,36579.53,2017-02-05,2018-10-11,1972-04-02,2.814568,44,9.396035e+04,1,672.666667,1
2,15706552,France,Male,0,1,156731.91,2016-02-15,2019-10-02,1979-09-09,3.789263,36,8.531170e+04,1,624.666667,0
3,15785358,Germany,Male,1,1,79366.98,2016-09-17,2016-12-20,1969-11-11,3.200613,46,1.069690e+05,1,742.166667,1
4,15594084,France,Male,1,0,74405.34,2016-03-17,2019-09-07,1993-06-08,3.704388,22,4.547474e-13,2,609.166667,0


We need to transform the categorial data to numerical

In [5]:
# Get dummy variable for Gender column
df_gender_dummy = pd.get_dummies(df["Gender"])
df = pd.concat([df_gender_dummy, df], axis=1)
df.drop("Gender", axis = 1, inplace=True)

In [6]:
df.head()

,Female,Male,CustomerId,Geography,HasCrCard,IsActiveMember,EstimatedSalary,application_date,exit_date,birth_date,years_company,age,balance_bank,Products,Score,Desertion
0,1,0,15610711,Germany,0,0,167673.37,2017-03-05,2019-11-27,1976-10-21,2.737907,40,1.286445e+05,1,644.500000,0
1,0,1,15764170,Germany,1,0,36579.53,2017-02-05,2018-10-11,1972-04-02,2.814568,44,9.396035e+04,1,672.666667,1
2,0,1,15706552,France,0,1,156731.91,2016-02-15,2019-10-02,1979-09-09,3.789263,36,8.531170e+04,1,624.666667,0
3,0,1,15785358,Germany,1,1,79366.98,2016-09-17,2016-12-20,1969-11-11,3.200613,46,1.069690e+05,1,742.166667,1
4,0,1,15594084,France,1,0,74405.34,2016-03-17,2019-09-07,1993-06-08,3.704388,22,4.547474e-13,2,609.166667,0


# Modeling & Evaluation

Select the feactures and label

In [7]:
y = df['Desertion'].values
y

array([0, 1, 0, ..., 1, 0, 0])

In [8]:
X = df[['HasCrCard','IsActiveMember','EstimatedSalary','age','balance_bank','Products','Score','Female','Male']]
X.head()

,HasCrCard,IsActiveMember,EstimatedSalary,age,balance_bank,Products,Score,Female,Male
0,0,0,167673.37,40,1.286445e+05,1,644.500000,1,0
1,1,0,36579.53,44,9.396035e+04,1,672.666667,0,1
2,0,1,156731.91,36,8.531170e+04,1,624.666667,0,1
3,1,1,79366.98,46,1.069690e+05,1,742.166667,0,1
4,1,0,74405.34,22,4.547474e-13,2,609.166667,0,1


Let's normalize the Data

In [9]:
X = preprocessing.StandardScaler().fit(X).transform(X)
X

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  if __name__ == '__main__':


array([[-1.54818408, -1.03057628,  1.17508595, ..., -0.12986584,
         1.09590728, -1.09590728],
       [ 0.64591802, -1.03057628, -1.10446019, ...,  0.58240672,
        -0.91248595,  0.91248595],
       [-1.54818408,  0.9703309 ,  0.98482863, ..., -0.63140687,
        -0.91248595,  0.91248595],
       ...,
       [ 0.64591802, -1.03057628,  1.63079455, ...,  0.12722663,
         1.09590728, -1.09590728],
       [-1.54818408, -1.03057628,  1.02172858, ...,  1.16824036,
         1.09590728, -1.09590728],
       [ 0.64591802, -1.03057628,  1.3424092 , ..., -1.41954278,
        -0.91248595,  0.91248595]])

Let's select the traning set and test set

In [10]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (6997, 9) (6997,)
Test set: (3000, 9) (3000,)


Neural network

In [11]:
classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal', input_dim=9))
#Second  Hidden Layer
classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal'))
#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
#Compiling the neural network
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [13]:
#Fitting the data to the training dataset
classifier.fit(X_train,y_train, batch_size=10, epochs=50)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
6997/6997 [==============================] - 12s 2ms/step - loss: 0.5082 - acc: 0.7951
Epoch 2/50
6997/6997 [==============================] - 10s 1ms/step - loss: 0.4290 - acc: 0.7951
Epoch 3/50
6997/6997 [==============================] - 10s 1ms/step - loss: 0.4244 - acc: 0.7951
Epoch 4/50
6997/6997 [==============================] - 10s 1ms/step - loss: 0.4197 - acc: 0.7951
Epoch 5/50
6997/6997 [==============================] - 11s 2ms/step - loss: 0.4149 - acc: 0.8138: 2s 
Epoch 6/50
6997/6997 [==============================] - 11s 2ms/step - loss: 0.4098 - acc: 0.8236
Epoch 7/50
6997/6997 [==============================] - 11s 2ms/step - loss: 0.4044 - acc: 0.8244
Epoch 8/50
6997/6997 [==============================] - 10s 1ms/step - loss: 0.3986 - acc: 0.8265
Epoch 9/50
6997/6997 [==============================] - 11s 2ms/step - loss: 0.3935 - acc: 0.8274
Epoch 10/50
6997/6997 [==============================] - 9s 1ms/s

In [14]:
score = classifier.evaluate(X_test, y_test)

3000/3000 [==============================] - 1s 188us/step


In [15]:
print ('Accuracy:',score[1]) 

Accuracy: 0.8599999998410542
